In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.pipeline import Pipeline
import seaborn as sns
import os
import folium
import geopandas as gpd
import scipy.stats as stat

# nettoyage et aménagement des données expérimentales

path='ANSUR.csv'
data=pd.read_csv(path,engine='python',sep=';')
data['Height in']=data['Height in']*0.0254 # conversion de l'unité de mesure d'hauteur ("pouces" -> "mètres")
data['Weight lbs']=data['Weight lbs']*0.453592 # conversion de l'unité de mesure ("livres" -> "kilogrammes")

data['Ethnicity'].replace(np.nan, "american", inplace=True)



# nettoyage et amnégament des données spatiales, pour l'étape suivante

path_geo='localisations_pays.csv'
data_geo_USA=pd.read_csv(path_geo, sep=';')[['usa_state','usa_state_latitude','usa_state_longitude']]
data_geo_USA.sort_values('usa_state')
data_geo_USA['geometry']=[f'POINt ({i} {j})' for i,j in zip (data_geo_USA['usa_state_latitude'],data_geo_USA['usa_state_longitude']) ]





#cartographie des caractéristiques socio-démographiques ( Age, provenance, hauteur, poids)  de l'échantillon d'étude, à partir du dataframe "data"


data.sort_values(by=['Subjects Birth Location'],ascending=True, inplace=True)

data_socio=data[['Subjects Birth Location','Age','Height in','Weight lbs']].groupby(['Subjects Birth Location'],as_index=False).mean()
data_socio.sort_values(by=['Subjects Birth Location'],ascending=True, inplace=True)
data_socio['counts']=data_socio['Subjects Birth Location'].map(data['Subjects Birth Location'].value_counts(sort=True))


data_socio.rename( columns={ 'Subjects Birth Location' : 'usa_state'}, inplace=True)
merged=pd.merge(data_socio, data_geo_USA, on='usa_state',how='inner')


socio_map=folium.Map(location=[40,-96], zoom_start=4.0, tiles='Mapbox Bright', label='Carte des caractéristiques sociodémographiques de l échantillon d étude')
usa_geo=r'coordonnées_USA.json'


socio_map.choropleth(geo_data=usa_geo, data=merged, columns=['usa_state','counts'], key_on='feature.properties.NAME', fill_color='Blues', legend_name='provenance des candidats ( Source:  C., Churchill (1988):"Anthropometric Survey of U.S. Army Personnel: Summary Statistics Interim Report"')
for ix,row in  merged.iterrows():
  popup_data_socio=pd.DataFrame(data=[['Etat',str(row['usa_state'])],['Age moyen',f'{round(row["Age"],2) } ans'],['Hauteur moyenne', f'{ round(row["Height in"],2) } m'], ['Poids moyen', f'{ int(row["Weight lbs"]) } Kg']])
  print(popup_data_socio)
  input('mm')
  popup_html=popup_data_socio.to_html(classes='table table-striped table-hover table-condensed table-responsive', index=False, header=False)
  folium.CircleMarker(location = [float(row['usa_state_latitude']),float(row['usa_state_longitude'])], radius=2, popup=folium.Popup(popup_html), color='#0000FF', fill_color='#0000FF').add_to(socio_map)

socio_map.save('macarte.html')







 #modélisation de l'ensemble des correlations entre les variables mesurées de l'expérience, via une "heatmap"
data1=data._get_numeric_data().astype('int64')       # dataframe utilisé pour l'ensemble des opérations de modélisations prédictive



input1=input('ne validez aucune touche pour continuer sur heatmap')
corr=data1.corr()
if input1=='':
  plt.figure(figsize=(90,90))
  ax=sns.heatmap(corr,cmap=sns.diverging_palette(20, 220, n=200))
  ax.set_xticklabels(data1.columns,rotation=90, fontsize=4)
  ax.set_yticklabels(data1.columns, fontsize=4)
  ax.set_title('Carte des correlations entre les différents paramètres physiques et socio-démgraphiques chez les sujets \n (Source: "Anthropometric Survey of U.S. Army Personnel: "Summary Statistics Interim Report"(1988) )', fontsize=12)
  plt.show()




# modélisation prédictive de la Vd "poids", selon un modèle de régression linéaire et une équation polynomiale unidimensionnelle

lr=LinearRegression()
def graph_lr_simple(X_data, Y_data, Nomcourbe1, Nomcourbe2) :

    x_train, x_test, y_train, y_test=train_test_split(X_data, Y_data, test_size=0.45, random_state=0)
    lr.fit(x_train, y_train)
    Yhat=lr.predict(x_test)

    plt.figure()
    ax1= sns.distplot(Yhat,hist='False',color='g',label=Nomcourbe1)
    sns.distplot(y_test, hist='False', color='b', label=Nomcourbe2, ax=ax1)
    plt.legend()
    plt.grid()
    plt.xlabel(f'poids (10^-1.Kg),coefficient de détermination R^2:{lr.score(x_test, y_test)} ',fontsize=8)
    plt.ylabel(f"proportion d'individus", fontsize=6)
    plt.title(f'Distribution des fonctions prédites et réelles, établis entre la VD: { y_test.name} \n et les VI: {x_test.columns.values}', fontsize=9 )
    plt.show()
    plt.close()
    
 

graph_lr_simple(data1[['buttock circumference','hip breadthsitting','thigh circumference', 'chest circumference' ,'vertical trunk circumference', 'waist circumference']], data1['weight kg'],'données prédites', 'données récoltées')




# modélisation prédictive de la Vd "poids", ce en fonction des variables :('bideltoid breadth','anklec ircumference','buttock circumference','calf circumference', 'chestc ircumference', 'hip breadthsitting', 'lower thigh circumference', 'shoulder circumference'  )

def graph_pr_à_1_dimension (X_data, Y_data,  Nomcourbe1, Nomcourbe2, n):
  degres= np.arange(1,n,1)
  x_train, x_test, y_train, y_test=train_test_split(X_data, Y_data, test_size=0.20, random_state=5)

  lr=LinearRegression()

  list_R2=[]
  list_MAE=[]
  list_MSE=[]
  list_variance=[]

  for n in degres:
   
    input= [('scale',StandardScaler()),('polynomial',PolynomialFeatures(degree=n)),('mode',LinearRegression())]
    pipe=Pipeline(input)
    pipe.fit(x_train,y_train)
    Yhat=pipe.predict(x_test)
    list_R2.append(pipe.score(x_test, y_test))
    list_MAE.append(mean_absolute_error(y_test,Yhat))
    list_MSE.append(mean_squared_error(y_test,Yhat))
    list_variance.append(y_train.var())

  
  print(y_test.index)
  print(x_test.columns)
  fig, ax=plt.subplots(2,1,sharex=True)
  ax[0].plot(degres, list_R2,color='R',label="R^2")
  ax[0].set_xlabel('degres',fontsize=6)
  ax[0].set_ylabel(f'scores,  R^2 maximale: {max(list_R2)}, degré optimale: {list_R2.index(max(list_R2))}', fontsize=5 )
  ax[0].set_title(f'évolution du R^2 en fonction des degrées polynomiaux, \n entre la VI: {y_test.name} et les VD: {x_test.columns.values} ', fontsize=8)
  ax[0].set_ylim(-10,2)
  plt.legend()

  
  ax[1].plot(degres, list_MSE, color='g', label='MSE')
  ax[1].plot(degres, list_MAE, color='b', label='MAE')
  ax[1].set_xlabel('degres', fontsize=5)
  ax[1].set_ylabel(f'scores',fontsize=5)
  ax[1].set_title('évolution des indices d erreur en fonction des degrées polynomiaux')
  ax[1].set_ylim(0,10000)
  plt.legend()
  plt.show()

graph_pr_à_1_dimension (data1[['buttock circumference','hip breadthsitting','thigh circumference', 'chest circumference' ,'vertical trunk circumference', 'waist circumference']], data1['weight kg'] , 'données prédites','données récoltées', 10)




'bideltoid breadth','buttock circumference','ankle circumference','buttock circumference','calf circumference','waist circumference','lower thigh circumference'
# variables les plus influentes: 'buttock circumference','hip breadthsitting', 'chest circumference' ,'vertical trunk circumference', 'waist circumference'
